        Ситуация:

У меня открыто 2 брокерских счета у Finam и Тинькофф.

Мне необходимо самостоятельно отчитываться прибыли/убытках от продажи валюты.

Расходы на покупку валюты необходимо рассчитать методом FIFO (First In First Out), то есть при продаже валюты, происходит продажа валюты, которая куплена раньше всего.

        Задача:

Для корректного расчета расходов на покупку валюты требуется поочередно обработать все сделки с валютой у брокера за все время.

Расчеты нужно произвести для каждого брокера отдельно.

        Действия:

Я написал три пользовательские функции:

- tinkoff_def - для обработки отчетов Тинькофф брокера (может быть использована без других функций)

- finam_def - для обработки отчетов брокера Finam (может быть использована без других функций)

- sell_def - функция для расчета прибыли/убытков от проведенных сделок (принимает на обработку датафрейм, полученный от работы функций tinkkoff_def или finam_def)

        Результат:

В дальнейшем расчет прибыли/убытков от операций на фондовом рынке не будет занимать значительного времени, так как расчеты автоматизированы.

Для каждой функции написана документация, код содержит пояснения к коду.

In [18]:
import pandas as pd
import re
import os
import warnings


warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.set_option('display.max_columns', 40)

# функция для обработки отчетов Тинькофф


def tinkoff_def(folder: str):
    '''
    Функция обрабатывает отчеты брокера Тинькофф.\n
    Формат отчетов: .xlsx образца 2022 г.\n
    Параметры:\n
    folder: str\n
        Папка с отчетами брокера (отдельная папка для брокера)\n
    Пример:\n
    tinkoff_def('C:/Projects/Налоги/Тинькофф/')
    '''
    # указываем путь к папке с отчетами
    tinkoff_folder = folder
    tinkoff_files = os.listdir(tinkoff_folder)
    # задаем регулярное выражение для выделения сделок с валютой
    match = re.compile(r'_TOM$|_TOD$')

    # создаем пустой датафрейм для отчетов
    tinkoff_dfs = pd.DataFrame()
    # цикл для обработки файлов в папке
    for file in tinkoff_files:
        # Читаем файл Excel
        df = pd.read_excel(tinkoff_folder + file, skiprows=7).dropna(axis=0, how='all').dropna(axis=1, how='all')

        # Убираем лишние строки
        df_last_ind = int(
            df
            [df.isin(['1.2 Информация о неисполненных сделках на конец отчетного периода']).any(
                axis=1)].index[0])
        df = df[0:df_last_ind]

        # Первая часть таблицы
        df1 = df[~df['Номер сделки'].isna()].dropna(axis=1, how='all').dropna(axis=0, how='all')
        df1 = df1[df1['Номер сделки'] != 'Номер сделки']
        df1_ind = pd.Index(range(0, df1.reset_index()['index'].count(), 1))
        df1 = df1.set_index(df1_ind)

        # удаляем спец. символы и лишние пробелы
        col_list1 = list(pd.Series(df1.columns).replace('\n', '', regex=True))
        col_list1_strip = [col.strip() for col in col_list1]
        df1.columns = col_list1_strip

        # оставляем нужные столбцы
        df1 = df1[['Номер сделки', 'Дата заключения', 'Время', 'Вид сделки', 'Сокращенное наименование',
                   'Цена за единицу', 'Количество', 'Сумма сделки', 'Валюта расчетов', 'Дата расчетов']]
        # выбираем сделки с валютой
        df1 = df1[df1['Сокращенное наименование'].str.contains(match) == True]

        # Вторая часть таблицы
        df2 = df[df[df.columns[0]].isna()].dropna(axis=1, how='all').dropna(axis=0, how='all')
        df2 = df2[~df2[df2.columns[0]].isna()]
        # присваиваем имена столбцам, удаляем спец. символы и лишние пробелы
        col_list2 = list(df2.iloc[0].astype(str).replace('\n', '', regex=True))
        col_list2_strip = [col.strip() for col in col_list2]
        df2.columns = col_list2_strip

        df2 = df2[1:]
        df2 = df2.dropna(axis=1, how='all').dropna(axis=0, how='all')

        # оставляем нужные столбцы
        df2 = df2[['Номер сделки', 'Дата заключения', 'Время', 'Вид сделки', 'Сокращенное наименование',
                   'Цена за единицу', 'Количество', 'Сумма сделки', 'Валюта расчетов', 'Дата расчетов']]
        # выбираем сделки с валютой
        df2 = df2[df2['Сокращенное наименование'].str.contains(match) == True]
        # объединяем таблицы
        df_t = pd.concat([df1, df2], ignore_index=True)
        # изменяем тип данных для числовых данных

        df_t['Дата заключения'] = pd.to_datetime(
            df_t[['Дата заключения', 'Время']].agg(' '. join, axis=1), dayfirst=True)
        df_t['Дата расчетов'] = pd.to_datetime(df_t['Дата расчетов'], dayfirst=True)
        df_t[['Цена за единицу', 'Сумма сделки']] = df_t[['Цена за единицу',
                                                          'Сумма сделки']].replace(',', '.', regex=True)

        df_t = (
            df_t
            .drop('Время', axis=1)
            .astype(
                {
                    'Количество': 'int32',
                    'Цена за единицу': 'float32',
                    'Сумма сделки': 'float32'
                }
            ))
        # Объединяем таблицы, полученные при работе цикла
        tinkoff_dfs = pd.concat([tinkoff_dfs, df_t], ignore_index=True)
        print(f'{file} обработан')
    tinkoff_dfs['Сокращенное наименование'] = tinkoff_dfs['Сокращенное наименование'].replace(match, '', regex=True)
    return (tinkoff_dfs)


# функция для обработки отчетов Finam
def finam_def(folder: str):
    '''
    Функция обрабатывает отчеты брокера Finam.\n
    Формат отчетов: .html образца 2022 г.\n
    Параметры:\n
    folder: str\n
        Папка с отчетами брокера (отдельная папка для брокера)\n
    Пример:\n
    finam_def('C:/Projects/Налоги/Finam/')
    '''
    # указываем путь к папке с отчетами
    finam_folder = folder
    finam_files = os.listdir(finam_folder)
    # задаем регулярное выражение для выделения сделок с валютой
    html_match = re.compile(r'_TOM|_TOD')

    # создаем пустой датафрейм для отчетов
    finam_dfs = pd.DataFrame()
    # цикл для обработки файлов в папке
    for file in finam_files:
        # условие для исключения обработки лишних файлов
        if 'html' not in file:
            continue
        # код для обработки файлов
        try:
            # читаем файл
            df = pd.read_html(finam_folder + file, match=html_match, thousands=' ')
            # выбираем таблицу
            df = df[0]
            # выбираем нужные столбцы
            df = df[['№ сделки', 'Дата сделки', 'Время сделки', 'Вид сделки', 'Инструмент', 'Цена',
                     'Количество (шт.)', 'Объём сделки', 'Валюта котирования', 'Дата исполнения (факт)']]
            # переименование столбцов
            df.columns = ['Номер сделки', 'Дата заключения', 'Время', 'Вид сделки', 'Сокращенное наименование',
                          'Цена за единицу', 'Количество', 'Сумма сделки', 'Валюта расчетов', 'Дата расчетов']
            # оставляем только операции с валютой
            df = df[df['Сокращенное наименование'].str.contains(html_match) == True]
            # исправляем тип данных
            df['Количество'] = df['Количество'].fillna(0).astype('int32')
            df[['Цена за единицу', 'Сумма сделки']] = df[['Цена за единицу', 'Сумма сделки']].replace(
                ',', '.', regex=True).replace(' ', '', regex=True).fillna(0).astype('float32')
            df['Валюта расчетов'] = df['Валюта расчетов'].replace('Рубль', 'RUB')
            df['Дата заключения'] = pd.to_datetime(df[['Дата заключения', 'Время']].agg(' '. join, axis=1))
            df['Дата расчетов'] = pd.to_datetime(df['Дата расчетов'], format='%d.%m.%Y')
            df = df.drop('Время', axis=1)
            # объединяем полученные дата фреймы
            finam_dfs = pd.concat([finam_dfs, df], ignore_index=True)
            print(f'Файл {file} обработан')
        # продолжение цикла, если в файле нет нужный данных
        except ValueError as e:
            print(f'В файле {file} нет нужной таблицы!\n ValueError:', str(e))
            continue
    finam_dfs['Сокращенное наименование'] = finam_dfs['Сокращенное наименование'].replace(html_match, '', regex=True)
    return finam_dfs


# функция для расчета доходов и расходов по методу FIFO
def sell_def(folder: str, year: int):
    '''
    Warning: использовать только при совершении покупки и продажи валюты в рамках одного брокерского счета!\n
    Функция рассчитывает доходы/убытки от операций с валютой по методу FIFO.\n
    Расчеты должны производиться для кажного брокера отдельно.\n
    Форматы отчетов:\n
        Тинькофф - .xlsx образца 2022 г.\n
        Finam - .html образца 2022 г.\n
    Параметры:\n
    folder: str\n
        Папка с отчетами брокера (отдельная папка для брокера)\n
        Папка с отчетами должна содержать имя брокера: "Тинькофф" или "Finam".\n
    year: int\n
        Отчетный год\n
    Пример:\n
    sell_def('C:/Projects/Налоги/Finam/', 2022)\n
    sell_def('C:/Projects/Налоги/Тинькофф/', 2022)
    '''

    if 'finam' in folder.lower():
        df = finam_def(folder)
    elif 'тинькофф' in folder.lower():
        df = tinkoff_def(folder)

    # создаем лист с валютами
    currency_list = list(df['Сокращенное наименование'].unique())

# создаем пустые датафреймы для отчетов
    all_buy = pd.DataFrame()
    all_sell = pd.DataFrame()

# цикл для последовательного расчета стоимости покупок по методу FIFO для разных валют
    for currency in currency_list:
        # фильтруем датафрейм по валюте
        currency_df = df[df['Сокращенное наименование'] == currency]

    # создание таблицы с покупками
        df_buy = currency_df[currency_df['Вид сделки'] == 'Покупка'].copy()
    # индексы (с 0 до n) для таблицы
        ind_buy = pd.Index(range(0, df_buy.reset_index()['index'].count(), 1))
    # обновление индексов для таблицы с покупками
        df_buy = df_buy.set_index(ind_buy)

    # создание таблицы с продажами
        df_sell = currency_df[currency_df['Вид сделки'] == 'Продажа'].copy()
    # индексы (с 0 до n) для таблицы
        ind_sell = pd.Index(range(0, df_sell.reset_index()['index'].count(), 1))
    # обновление индексов для таблицы с продажами
        df_sell = df_sell.set_index(ind_sell)

    # создание столбца для себестоимости
        df_sell["Себестоимость покупки"] = 0
    # копия столбца с количеством проданных единиц
        df_sell["Sell_cnt"] = df_sell['Количество']

    # обрабатываем данные из таблиц
    # задаем изначальные значения индесков i, j для перебора строк, переменной для хранения стоимости покупок Buy_cost
        i = 0
        j = 0
        Buy_cost = 0
    # листы с индексами датафреймов
        index_sell = tuple(df_sell.index)
        index_buy = tuple(df_buy.index)
    # Обрабатываем датафреймы с покупками и продажами построчно. Цикл ограничен размерами массивов
        while i in index_sell and j in index_buy:
            # в цикле сравниваются 1 сделка по покупке и продаже валюты
            if df_sell.at[i, 'Sell_cnt'] > df_buy.at[j, 'Количество']:
                # количество проданных единиц больше количества купленных
                Buy_cost += df_buy.at[j, 'Количество']*df_buy.at[j, 'Цена за единицу']
                df_sell.at[i, 'Sell_cnt'] -= df_buy.at[j, 'Количество']
                df_buy.at[j, 'Количество'] = 0
                j += 1
            elif df_sell.at[i, 'Sell_cnt'] == df_buy.at[j, 'Количество']:
                # количество проданных единиц равно количеству купленных
                Buy_cost += df_buy.at[j, 'Количество']*df_buy.at[j, 'Цена за единицу']
                df_buy.at[j, 'Количество'] = 0
                df_sell.at[i, "Себестоимость покупки"] = round(Buy_cost, 2)
                Buy_cost = 0
                j += 1
                i += 1
            else:
                # количество проданных единиц меньше количества купленных
                Buy_cost += df_sell.at[i, 'Sell_cnt']*df_buy.at[j, 'Цена за единицу']
                df_buy.at[j, 'Количество'] -= df_sell.at[i, 'Sell_cnt']
                df_sell.at[i, "Себестоимость покупки"] = round(Buy_cost, 2)
                Buy_cost = 0
                i += 1

    # объединение полученных датафреймов в один
        all_buy = pd.concat([all_buy, df_buy], ignore_index=True)
        all_sell = pd.concat([all_sell, df_sell], ignore_index=True)

# удаляем лишний столбец
    all_sell = all_sell.drop('Sell_cnt', axis=1)

# фильтруем таблицу по выбранному году
    sell_year = all_sell[(all_sell['Дата расчетов'] >= f'{year}-01-01')
                         & (all_sell['Дата расчетов'] <= f'{year}-12-31')].copy()

# считаем прибыль/убыток от сделок
    sell_year['Прибыль_убыток'] = sell_year['Сумма сделки'] - sell_year['Себестоимость покупки']

    return sell_year


In [2]:
import gdown

tinkoff_files_folder = 'https://drive.google.com/drive/folders/1azEai60SwVfN5p0wVo3bM_yQvGEZDYvG?usp=sharing'
finam_files_folder = 'https://drive.google.com/drive/folders/1I0z962kPezPbGi7wd1uUTaWki46XPb0J?usp=sharing'

gdown.download_folder(tinkoff_files_folder, quiet=False, remaining_ok=True)
gdown.download_folder(finam_files_folder, quiet=False, remaining_ok=True)


Retrieving folder list


Processing file 1KR6faJd2b5yoGIToq3ot-TtMrEL_ecKj broker-report-2019-09-01-2019-12-31.xlsx
Processing file 1qceBHxUieoS_CfUpLtDDzOhVHQ-y2i-Z broker-report-2020-01-01-2020-12-31.xlsx
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1KR6faJd2b5yoGIToq3ot-TtMrEL_ecKj
To: e:\Поиск работы\Портфолио\FIFO calculation\Тинькофф\broker-report-2019-09-01-2019-12-31.xlsx
100%|██████████| 30.5k/30.5k [00:00<?, ?B/s]
Downloading...
From: https://drive.google.com/uc?id=1qceBHxUieoS_CfUpLtDDzOhVHQ-y2i-Z
To: e:\Поиск работы\Портфолио\FIFO calculation\Тинькофф\broker-report-2020-01-01-2020-12-31.xlsx
100%|██████████| 772k/772k [00:00<00:00, 3.85MB/s]
Download completed
Retrieving folder list


Processing file 1-Hlw8n65hl2QK3hvv4oVGilCt7WhD9M_ Черновол Денис Владимирович КЛФ-1463051 (01.10.2022 по 31.12.2022)_new.html
Processing file 1dgctD8D6jAMMrorGGdvlYw5064VmoPzV Черновол Денис Владимирович КЛФ-1463051 (06.07.2022 по 30.09.2022)_new.html
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1-Hlw8n65hl2QK3hvv4oVGilCt7WhD9M_
To: e:\Поиск работы\Портфолио\FIFO calculation\Finam\Черновол Денис Владимирович КЛФ-1463051 (01.10.2022 по 31.12.2022)_new.html
100%|██████████| 8.34k/8.34k [00:00<?, ?B/s]
Downloading...
From: https://drive.google.com/uc?id=1dgctD8D6jAMMrorGGdvlYw5064VmoPzV
To: e:\Поиск работы\Портфолио\FIFO calculation\Finam\Черновол Денис Владимирович КЛФ-1463051 (06.07.2022 по 30.09.2022)_new.html
100%|██████████| 74.6k/74.6k [00:00<00:00, 74.5MB/s]
Download completed


['e:\\Поиск работы\\Портфолио\\FIFO calculation\\Finam\\Черновол Денис Владимирович КЛФ-1463051 (01.10.2022 по 31.12.2022)_new.html',
 'e:\\Поиск работы\\Портфолио\\FIFO calculation\\Finam\\Черновол Денис Владимирович КЛФ-1463051 (06.07.2022 по 30.09.2022)_new.html']

In [19]:
finam_sell = sell_def('E:/Поиск работы/Портфолио/FIFO calculation/Finam/new_files/', 2022)


В файле Черновол Денис Владимирович КЛФ-1463051 (01.10.2022 по 31.12.2022)_new.html нет нужной таблицы!
 ValueError: No tables found matching pattern '_TOM|_TOD'
Файл Черновол Денис Владимирович КЛФ-1463051 (06.07.2022 по 30.09.2022)_new.html обработан


In [20]:
finam_sell


,Номер сделки,Дата заключения,Вид сделки,Сокращенное наименование,Цена за единицу,Количество,Сумма сделки,Валюта расчетов,Дата расчетов,Себестоимость покупки,Прибыль_убыток
0,513268000.0,2022-01-08 17:16:28,Продажа,USDRUB,6038.0,9000,543420.0,RUB,2022-08-01,5532075000,-5.531532e+09
1,513402000.0,2022-02-08 10:02:22,Продажа,USDRUB,602425.0,1000,60242.5,RUB,2022-08-02,614675000,-6.146148e+08
2,513425000.0,2022-02-08 10:07:44,Продажа,USDRUB,6010.0,5000,300500.0,RUB,2022-08-02,3011125000,-3.010824e+09
3,513612000.0,2022-02-08 11:24:45,Продажа,USDRUB,601225.0,10000,601225.0,RUB,2022-08-02,6013750000,-6.013149e+09
4,513616000.0,2022-02-08 11:26:28,Продажа,USDRUB,601475.0,10000,601475.0,RUB,2022-08-03,60140000,-5.953852e+07
5,513403000.0,2022-02-08 10:03:03,Продажа,CNYRUB,8986.0,60000,539160.0,RUB,2022-08-02,538740000,-5.382008e+08
6,523729000.0,2022-01-09 10:11:47,Продажа,CNYRUB,8726.0,10000,87260.0,RUB,2022-09-02,89390000,-8.930274e+07


In [20]:
tinkoff_sell = sell_def('C:/Users/User/Projects/Налоги Python/Тинькофф/', 2022)


broker-report-2019-09-01-2019-12-31.xlsx обработан
broker-report-2020-01-01-2020-12-31.xlsx обработан
broker-report-2021-01-01-2021-12-31.xlsx обработан
broker-report-2022-01-01-2022-12-31.xlsx обработан


In [22]:
finam_sell.groupby('Сокращенное наименование').agg(
    {'Сумма сделки': sum, 'Себестоимость покупки': sum})


,Сумма сделки,Себестоимость покупки
Сокращенное наименование,,
CNYRUB,626420.0,628130.01
USDRUB,2106862.5,2118562.49


In [23]:
sum(finam_sell['Сумма сделки']), sum(finam_sell['Себестоимость покупки'])


(2733282.5, 2746692.5)

In [24]:
tinkoff_sell.groupby('Сокращенное наименование').agg(
    {'Сумма сделки': sum, 'Себестоимость покупки': sum})


,Сумма сделки,Себестоимость покупки
Сокращенное наименование,,
CNYRUB,8.750000,8.64
EURRUB,15517.780273,20861.63
HKDRUB,5889.020020,5035.57
USDRUB,643346.062500,760659.71


In [25]:
sum(tinkoff_sell['Сумма сделки']), sum(tinkoff_sell['Себестоимость покупки'])


(664761.5808105469, 786565.5499999999)

In [23]:
finam_folder = 'E:/Поиск работы/Портфолио/FIFO calculation/Finam_1/new_files/'
finam_files = os.listdir(finam_folder)
# задаем регулярное выражение для выделения сделок с валютой
html_match = re.compile(r'_TOM|_TOD')

# создаем пустой датафрейм для отчетов
finam_dfs = pd.DataFrame()
# цикл для обработки файлов в папке
for file in finam_files:
    # условие для исключения обработки лишних файлов
    if 'html' not in file:
        continue
    # код для обработки файлов
    try:
        # читаем файл
        df = pd.read_html(finam_folder + file, match=html_match, thousands=' ')
        # выбираем таблицу
        df = df[0]
        # выбираем нужные столбцы
        df = df[['№ сделки', 'Дата сделки', 'Время сделки', 'Вид сделки', 'Инструмент', 'Цена',
                 'Количество (шт.)', 'Объём сделки', 'Валюта котирования', 'Дата исполнения (факт)']]
        # переименование столбцов
        df.columns = ['Номер сделки', 'Дата заключения', 'Время', 'Вид сделки', 'Сокращенное наименование',
                      'Цена за единицу', 'Количество', 'Сумма сделки', 'Валюта расчетов', 'Дата расчетов']
        # оставляем только операции с валютой
        df = df[df['Сокращенное наименование'].str.contains(html_match) == True]
        # исправляем тип данных
        df['Количество'] = df['Количество'].fillna(0).astype('int32')
        df[['Цена за единицу', 'Сумма сделки']] = df[['Цена за единицу', 'Сумма сделки']].replace(
            ',', '.', regex=True).replace(' ', '', regex=True).fillna(0).astype('float32')
        df['Валюта расчетов'] = df['Валюта расчетов'].replace('Рубль', 'RUB')
        df['Дата заключения'] = pd.to_datetime(df[['Дата заключения', 'Время']].agg(' '. join, axis=1))
        df['Дата расчетов'] = pd.to_datetime(df['Дата расчетов'], format='%d.%m.%Y')
        df = df.drop('Время', axis=1)
        # объединяем полученные дата фреймы
        finam_dfs = pd.concat([finam_dfs, df], ignore_index=True)
        print(f'Файл {file} обработан')
    # продолжение цикла, если в файле нет нужный данных
    except ValueError as e:
        print(f'В файле {file} нет нужной таблицы!\n ValueError:', str(e))
        continue
finam_dfs['Сокращенное наименование'] = finam_dfs['Сокращенное наименование'].replace(html_match, '', regex=True)


В файле Черновол Денис Владимирович КЛФ-1463051 (01.10.2022 по 31.12.2022)_new.html нет нужной таблицы!
 ValueError: No tables found matching pattern '_TOM|_TOD'
Файл Черновол Денис Владимирович КЛФ-1463051 (06.07.2022 по 30.09.2022)_new.html обработан


In [24]:
finam_dfs


,Номер сделки,Дата заключения,Вид сделки,Сокращенное наименование,Цена за единицу,Количество,Сумма сделки,Валюта расчетов,Дата расчетов
0,504861000.0,2022-11-07 13:59:06,Покупка,USDRUB,614675.0,10000,"614 675,00",RUB,2022-07-11
1,513268000.0,2022-01-08 17:16:28,Продажа,USDRUB,6038.0,9000,"543 420,00",RUB,2022-08-01
2,513268000.0,2022-01-08 17:17:45,Покупка,CNYRUB,8979.0,40000,"359 160,00",RUB,2022-08-02
3,513268000.0,2022-01-08 17:18:29,Покупка,CNYRUB,8979.0,20000,"179 580,00",RUB,2022-08-02
4,513402000.0,2022-02-08 10:02:22,Продажа,USDRUB,602425.0,1000,"60 242,50",RUB,2022-08-02
5,513403000.0,2022-02-08 10:03:03,Продажа,CNYRUB,8986.0,60000,"539 160,00",RUB,2022-08-02
6,513419000.0,2022-02-08 10:06:29,Покупка,USDRUB,602225.0,5000,"301 112,50",RUB,2022-08-02
7,513425000.0,2022-02-08 10:07:44,Продажа,USDRUB,6010.0,5000,"300 500,00",RUB,2022-08-02
8,513609000.0,2022-02-08 11:23:29,Покупка,USDRUB,601375.0,10000,"601 375,00",RUB,2022-08-02
9,513612000.0,2022-02-08 11:24:45,Продажа,USDRUB,601225.0,10000,"601 225,00",RUB,2022-08-02


In [14]:
finam_dfs


,Номер сделки,Дата заключения,Вид сделки,Сокращенное наименование,Цена за единицу,Количество,Сумма сделки,Валюта расчетов,Дата расчетов
0,504860752.0,2022-11-07 13:59:06,Покупка,USDRUB,61.467499,10000,614675.0,RUB,2022-07-11
1,513267602.0,2022-01-08 17:16:28,Продажа,USDRUB,60.380001,9000,543420.0,RUB,2022-08-01
2,513268201.0,2022-01-08 17:17:45,Покупка,CNYRUB,8.979000,40000,359160.0,RUB,2022-08-02
3,513268452.0,2022-01-08 17:18:29,Покупка,CNYRUB,8.979000,20000,179580.0,RUB,2022-08-02
4,513402281.0,2022-02-08 10:02:22,Продажа,USDRUB,60.242500,1000,60242.5,RUB,2022-08-02
5,513403103.0,2022-02-08 10:03:03,Продажа,CNYRUB,8.986000,60000,539160.0,RUB,2022-08-02
6,513419155.0,2022-02-08 10:06:29,Покупка,USDRUB,60.222500,5000,301112.5,RUB,2022-08-02
7,513425386.0,2022-02-08 10:07:44,Продажа,USDRUB,60.099998,5000,300500.0,RUB,2022-08-02
8,513608986.0,2022-02-08 11:23:29,Покупка,USDRUB,60.137501,10000,601375.0,RUB,2022-08-02
9,513611970.0,2022-02-08 11:24:45,Продажа,USDRUB,60.122501,10000,601225.0,RUB,2022-08-02
